# Maîtriser l’utilisation des serveurs de calcul
## Analyse des tâches
Le but de ce chapitre est d'approfondir [l'analyse des tâches](https://docs.computecanada.ca/wiki/Running_jobs/fr#Suivi_des_t.C3.A2ches) en attente, en cours et les tâches terminées.

## Tâches en attente
Pour afficher les tâches en attente, on utilise la commande [`squeue -t pending`](https://slurm.schedmd.com/squeue.html) :
* Pour vos tâches : `squeue -t pending -u $USER`
* Toutes les tâches : `squeue -t pending`
* Toutes les tâches triées par priorité descendante :
```
squeue -t pending --sort=-p --format="%.16i %.9p %.8u %.16a %.18P %.10l %.4C %.7m"
```
  * `%i` : identifiant numérique de la tâche
  * `%p` : priorité de la tâche (entre 0.0 et 1.0)
  * `%u` : nom d'utilisateur
  * `%a` : compte de calcul utilisé
  * `%P` : partition de la grappe
  * `%l` : temps demandé
  * `%C` : nombre de processeurs demandé
  * `%m` : mémoire demandée

Note : vos tâches sont généralement en compétition avec des [tâches de la même partition](https://docs.computecanada.ca/wiki/Job_scheduling_policies/fr#Pourcentages_des_n.C5.93uds_disponibles).
Ainsi, une tâche CPU n'a aucun impact sur le temps d'attente d'une tâche GPU.
Vous pouvez utiliser la commande suivante pour obtenir le détail de votre tâche, incluant la partition à laquelle elle est assignée :
```
scontrol show job -dd <jobid>
```

Une chose qu'il faut comprendre de la [politique d'ordonnancement](https://docs.computecanada.ca/wiki/Job_scheduling_policies/fr) :
* **La priorité de votre tâche dépend majoritairement de l'utilisation récente** de votre groupe de recherche
  * Pour le reste, la priorité dépend aussi du temps d'attente, mais dans une très faible proportion
* L'information concernant l'utilisation groupe est donnée par la commande [`sshare`](https://docs.computecanada.ca/wiki/Job_scheduling_policies/fr#Priorisation_selon_la_juste_part) :
```
sshare -l -A def-prof1_cpu -u prof1,grad2,postdoc3
```
  * Plus le `LevelFS` du groupe est élevé, plus la priorité est élevée
  * L'utilisation est considérée normale si `LevelFS` est autour de 1.0
  * Si `LevelFS` est proche de 0.0, la priorité du groupe sera basse
* L'utilisation récente perd de son importance avec le temps qui passe.
Ainsi, son importance est progressivement réduite avec un ratio réduisant l'impact de 50% à chaque 7 jours
  * Par conséquent, le `LevelFS` et la priorité augmentent à nouveau lorsque le groupe reste en attente

## Tâches actives
Pour afficher les tâches actives, on utilise la commande [`squeue -t running`](https://slurm.schedmd.com/squeue.html) :
* Pour vos tâches : `squeue -t running -u $USER`
* Toutes les tâches : `squeue -t running`
* Toutes les tâches actives triées selon le temps qui leur reste :
```
squeue -t running --sort=+M --format="%.14i %.18P %.10M %.4C %.7m %R"
```
  * `%M` : temps de calcul écoulé
  * `%R` : liste des noeuds alloués

Pendant que vos tâches sont actives, vous pouvez vous connecter aux noeuds de calcul correspondants afin de valider que l'exécution se passe bien :
* Connexion avec : `ssh noeud`
* Inspection avec `top`, `htop`, `nvtop` et/ou `nvidia-smi` :
  * Est-ce que vos processus s'exécutent avec un pourcentage de 100%?
  * Est-ce que vos processus parallèles s'exécutent avec un pourcentage de `n` * 100%, où `n` est le nombre de processeurs par tâche Slurm?
  * Est-ce que les GPUs sont utilisés à leur plein potentiel?
    * Note : il est préférable de regarder la consommation en Watts plutôt que le pourcentage affiché
  * Est-ce que le noeud de calcul semble pleinement utilisé?

Toutes ces informations devraient vous permettre d'ajuster votre script de soumission et d'optimiser l'exécution parallèle de votre programme.

## Tâches terminées
Lorsqu'une [tâche est complétée](https://docs.computecanada.ca/wiki/Running_jobs/fr#T.C3.A2ches_termin.C3.A9es), la commande la plus simple pour obtenir des informations sur cette tâche est la commande :
```
seff <jobid>
```
Autrement, c'est aussi possible d'utiliser la commande Slurm `sacct` :
```
sacct -j <jobid> --format=JobID,JobName,MaxRSS,Elapsed
```
Encore une fois, ces informations permettent d'ajuster vos scripts et de mieux déterminer vos besoins en ressources.

Ultimement, l'utilisation totale du groupe est disponible sur CCDB :
* https://ccdb.computecanada.ca/me/group_usage

En tenant compte de l'utilisation passée, il devient possible d'estimer ses besoins pour le [Concours d'allocation de ressources](https://www.computecanada.ca/page-daccueil-du-portail-de-recherche/acces-aux-ressources/concours-dallocation-des-ressources/?lang=fr).

#### Exercice - Analyse d'exécution d'un pipeline
* Lancer à nouveau le pipeline avec la commande `bash scripts/pipeline_nbody.sh`
  * Au besoin, relancer le pipeline si vos tâches se sont exécutées trop rapidement
* Surveiller vos tâches :
  * `squeue -t pending -u $USER`
  * `squeue -t running -u $USER`
* Surveiller toutes les tâches :
  * `squeue -t pending`
  * `squeue -t running`
* Trier les tâches selon un critère :
  * `squeue -t pending --sort=-p --format="%.16i %.9p %.8u %.16a %.18P %.10l %.4C %.7m"`
  * `squeue -t running --sort=+M --format="%.14i %.18P %.10M %.4C %.7m %R"`

* Trouver vos tâches récentes avec :
  * `sacct`
* Analyser quelques tâches complétées avec :
  * `seff <jobid>`
  * `sacct -j <jobid> --format=JobID,JobName,MaxRSS,Elapsed`